### ESMA 4016
### Feature Selection en Clasificacion Supervisada
### Edgar Acuna

Uso de feature_selection de scikit-learn,  del modulo feature selction de Arizona State University y del modulo skrebate para feature selection

I-Usando la prueba de Chi-Square para ver si hay independenia entre la feature y la clase. 

De preferencia los datos deben estar discretizados y ademas con dos clases es mejor

Personalmente no la recomiendo porque la priueba de Chisquare es aproximada y no muy robusta


In [31]:
#Chi-Square usando el  modulo de Feature selction de Scikit-learn
import pandas as pd
import numpy as np
import math as m
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, SelectPercentile, f_classif, mutual_info_classif
# load data
url= "http://academic.uprm.edu/eacuna/diabetes.dat"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:8]

In [32]:
# Funcion auxiliar para discretizar cualquier columna de un dataframe
def disc_col_ew(df,str,k,out):
    df1=df[str]
    bins=np.linspace(df1.min(), df1.max(),k)
    if out=="num":
        df1=pd.cut(df1,bins=bins,include_lowest=True, right=True,labels=False)
    else:
        bins[0]=float('-inf')
        bins[k-1]=float('inf')
        df1=pd.cut(df1,bins=bins,include_lowest=True, right=True)  
    return df1

In [33]:
# funcion auxiliar para detreminar el numero optimo de intervalos segun la formula de scott
def nclass_scott(x):
    h=3.5*(np.var(x,ddof=1)**.5)*len(x)**(-.3333)
    intervals=m.ceil((max(x)-min(x))/h)
    return int(intervals)

In [34]:
#Funcion para discretizar todas las colunan de un dataframe
def disc_ew(df,out):
    name=df.columns.tolist()
    disc=pd.DataFrame()
    for name in df.columns.tolist():
        k=nclass_scott(df[name])
        disc[name]=disc_col_ew(df,name,k,out)
    return disc  

In [35]:
#discretizando las columnas de la matriz predictora X de diabetes
diab_disc=disc_ew(X,out="num")

In [36]:
y1=y.as_matrix()
X1=diab_disc.as_matrix()
# feature extraction
test = SelectKBest(score_func=chi2, k=3)
fit = test.fit(X1, y1)
# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)
features= fit.transform(X1)
# Imprime los datos de las tres mejores features
print(features)

[ 105.387  120.206    2.87    10.086   59.232   42.746   62.639  115.924]
[[ 4 11  6]
 [ 0  6  2]
 [ 6 14  2]
 ..., 
 [ 3  9  1]
 [ 0 10  5]
 [ 0  7  0]]


Comentario: Las tres mejores variables con la prueba de Chi-square son plas, age y preg por tener el Chi-Square mas alto.

II-Usando la prueba de estadistica de F para comparar dos o mas grupos

Se puede usar la libreria sciki-learn o la libreria de la ASU

In [37]:
# Selecion  de Features usando los p-values de la F-test como score
# Aqui usamos sciji-learn y se selecciona el 30% de deatures con el mas alro score
selector = SelectPercentile(f_classif, percentile=30)
fit=selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
print(scores)
features= fit.transform(X)
# Imprime los datos de las tres mejores features
print(features)

[ 0.221  1.     0.027  0.034  0.084  0.378  0.14   0.253]
[[ 148.    33.6   50. ]
 [  85.    26.6   31. ]
 [ 183.    23.3   32. ]
 ..., 
 [ 121.    26.2   30. ]
 [ 126.    30.1   47. ]
 [  93.    30.4   23. ]]


Comentario: Las tres, mejores variables con la prueba de F son plas,mass y age

In [38]:
#Usando la libreria Feature selection de la ASU
features, labels = X.values, data['class'].values
X_train, X_test, y_train, y_test = train_test_split(features, labels)
from skfeature.function.statistical_based import f_score
scoref = f_score.f_score(X_train, y_train)
idx = f_score.feature_ranking(scoref)
print(scoref,idx)

(array([  33.622,  169.293,    1.286,    5.408,    8.295,   58.266,
         12.778,   36.06 ]), array([1, 5, 7, 0, 6, 4, 3, 2], dtype=int64))


Comentario: las tres variables mas importantes con la prueba de F son: plas,mass y age

III- Usando Mutual Information (Entropia)

Libreria scikit-learn


In [39]:
# Univariate feature selection with Mutual Information
scores = mutual_info_classif(X,y)
scores /= scores.max()
print(scores)

[ 0.036  1.     0.041  0.008  0.322  0.649  0.073  0.433]


Comentario: Las tres variables usando el criterio de Mutual Information  son: plas, mass y age

IV-Usando ReliefF de la libreria skrebate

In [40]:
from skrebate import ReliefF
from sklearn.model_selection import train_test_split

df=data.drop('class',axis=1)
#Normalizando las predictoras
df_norm=(df - df.min()) / (df.max() - df.min())
features, labels = df_norm.values, data['class'].values

In [41]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

fs = ReliefF(n_neighbors=10)
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(df.columns,fs.feature_importances_):
    print (feature_name, feature_score)


('preg', 29.500000000000011)
('plas', 85.640703517587994)
('pres', 14.201754385964897)
('skin', 35.717171717171709)
('test', 16.226359338061467)
('mass', 42.022354694485877)
('pedi', 15.458795900939359)
('age', 27.358333333333324)


Comentario: las tres variables mas importantes con el ReliefF son: plas,mass y skin

In [44]:
#Usando la libreria feature selection de la ASU
from skfeature.function.similarity_based import reliefF
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  5.965,  16.247,   2.865,   6.428,   1.943,   6.396,   1.703,
         2.683]), array([1, 3, 5, 0, 2, 7, 4, 6], dtype=int64))


Comentario: las tres variables mas importantes con el RelieF son: plas, mass y skin

In [56]:
#Usando el Relief de Orange
import Orange
df = Orange.data.Table("diabetes")

def print_best_3(ma):
    for m in ma[:3]:
        print "%5.3f %s" % (m[1], m[0])

print 'Feature scores for best three features (scored individually):'
#ReliefF usando 10 vecinos mas cercanos y  una muestrd m-100 para updating de los pesos
meas = Orange.feature.scoring.Relief(k=10, m=100)
mr = [ (a.name, meas(a, df)) for a in df.domain.attributes]
mr.sort(key=lambda x: -x[1]) #sort decreasingly by the score
print_best_3(mr)

Feature scores for best three features (scored individually):
0.036 plas
0.026 age
0.015 mass


In [46]:
# Usando el reliefF de la ASU
#load data
url= "http://academic.uprm.edu/eacuna/bupa.dat"
names = ['mcv', 'alkphos', 'sgpt', 'aspar', 'gammagt', 'drinks', 'class']
data = pd.read_table(url, names=names,header=None)
y=data['class']
X=data.iloc[:,0:6]
y1=y.as_matrix()
X1=X.as_matrix()
features, labels = X.values, data['class'].values

In [47]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [48]:
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  11. ,  -64.8,  309.4,  122. ,   -8.6,   39.7]), array([2, 3, 5, 0, 4, 1], dtype=int64))


El Rielief de ASU recomienda  sgpt,aspar y drinks como las mejoras predictoras

In [50]:
fs = ReliefF(n_neighbors=10)
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(data.columns,fs.feature_importances_):
    print (feature_name, feature_score)


('mcv', 2.7631578947368438)
('alkphos', 0.37826086956521404)
('sgpt', 4.0933333333333373)
('aspar', 3.9480519480519489)
('gammagt', 5.0136986301369877)
('drinks', 4.1625000000000041)


El relielf de skrebate recomienda gammagt, sgpt y drinks 

In [53]:
url='http://academic.uprm.edu/eacuna/landsat.txt'
data = pd.read_table(url, header=None,delim_whitespace=True)
y=data.iloc[:,36]
X=data.iloc[:,0:37]
y1=y.as_matrix()
X1=X.as_matrix()
features, labels = X.values, y.values
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [54]:
#usando Relief de la ASU
score_relief=reliefF.reliefF(X_train,y_train)
feat=reliefF.feature_ranking(score_relief)
print(score_relief,feat)

(array([  930709.18 ,  1262010.886,   892029.445,  1049502.413,
         898223.814,  1153107.606,   823337.611,  1020532.51 ,
         958491.233,  1260176.102,   872225.923,  1188106.014,
         988273.692,  1302486.001,   888622.669,  1050787.881,
         952157.203,  1254166.666,   842781.857,  1005976.002,
         993024.781,  1307040.172,   792265.83 ,  1043069.516,
        1002251.517,  1342077.748,   845803.286,  1062645.522,
         934149.328,  1312146.663,   786174.152,   942112.38 ,
         950935.431,  1296243.801,   839812.149,  1102420.276,   220117.875]), array([25, 29, 21, 13, 33,  1,  9, 17, 11,  5, 35, 27, 15,  3, 23,  7, 19,
       24, 20, 12,  8, 16, 32, 31, 28,  0,  4,  2, 14, 10, 26, 18, 34,  6,
       22, 30, 36], dtype=int64))
